# Data preparation using Sklearn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Defining the column names based on the data description
cols = ['MPG', 'Cylinders', 'Displacements', 'Horsepower', 'Weight', 
        'Acceleration', 'Model Year', 'Origin']
# Reading the .data file using pandas
df = pd.read_csv('./auto-mpg.data', names=cols, na_values = "?",
                 comment = '\t', sep =" ", skipinitialspace=True)

# Making copy of the dataframe
data = df.copy()
data.head()

,MPG,Cylinders,Displacements,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [3]:
# Stratified split of the data
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data, data["Cylinders"]):
    train_set = data.loc[train_index]
    test_set = data.loc[test_index]

In [4]:
# Separating the target variable from the data
data = train_set.drop("MPG", axis=1)
data_labels = train_set["MPG"].copy()
data.head()


,Cylinders,Displacements,Horsepower,Weight,Acceleration,Model Year,Origin
145,4,83.0,61.0,2003.0,19.0,74,3
151,4,79.0,67.0,2000.0,16.0,74,2
388,4,156.0,92.0,2585.0,14.5,82,1
48,6,250.0,88.0,3139.0,14.5,71,1
114,4,98.0,90.0,2265.0,15.5,73,2


### Preprocessing the Origin Column



In [5]:
# function to map the numbers in the origin column to the country
def preprocess_origin_col(df):
    df_preprocess = df.copy()
    df_preprocess["Origin"]  = df_preprocess["Origin"].map({1: "India", 2: "USA",3: "Germany"})
    return df_preprocess
data_tr = preprocess_origin_col(data)
data_tr.head()




,Cylinders,Displacements,Horsepower,Weight,Acceleration,Model Year,Origin
145,4,83.0,61.0,2003.0,19.0,74,Germany
151,4,79.0,67.0,2000.0,16.0,74,USA
388,4,156.0,92.0,2585.0,14.5,82,India
48,6,250.0,88.0,3139.0,14.5,71,India
114,4,98.0,90.0,2265.0,15.5,73,USA


In [6]:
data_cat = data_tr[['Origin','Cylinders']]
data_cat.head()

,Origin,Cylinders
145,Germany,4
151,USA,4
388,India,4
48,India,6
114,USA,4


In [7]:
#One hot encoding categorical values
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
data_cat_1hot = cat_encoder.fit_transform(data_cat)
data_cat_1hot

<318x8 sparse matrix of type '<class 'numpy.float64'>'
	with 636 stored elements in Compressed Sparse Row format>

### Missing values

In [8]:
# Segregating numerical data
num_data = data.iloc[:,:-1]
num_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 145 to 362
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Cylinders      318 non-null    int64  
 1   Displacements  318 non-null    float64
 2   Horsepower     314 non-null    float64
 3   Weight         318 non-null    float64
 4   Acceleration   318 non-null    float64
 5   Model Year     318 non-null    int64  
dtypes: float64(4), int64(2)
memory usage: 17.4 KB


In [9]:
# Getting the missing values in Horsepower using the media
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
imputer.fit(num_data)

# Filing the missing values
X = imputer.transform(num_data)

# Converting the array back to a dataframe
data_tr = pd.DataFrame(X, columns = num_data.columns, index = num_data.index)

data_tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 145 to 362
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Cylinders      318 non-null    float64
 1   Displacements  318 non-null    float64
 2   Horsepower     318 non-null    float64
 3   Weight         318 non-null    float64
 4   Acceleration   318 non-null    float64
 5   Model Year     318 non-null    float64
dtypes: float64(6)
memory usage: 17.4 KB


### Adding new atributes 

In [10]:
num_data.head()

,Cylinders,Displacements,Horsepower,Weight,Acceleration,Model Year
145,4,83.0,61.0,2003.0,19.0,74
151,4,79.0,67.0,2000.0,16.0,74
388,4,156.0,92.0,2585.0,14.5,82
48,6,250.0,88.0,3139.0,14.5,71
114,4,98.0,90.0,2265.0,15.5,73


In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

# Index of the colums given their position in the dataframe
acceleration_ind = 4
horsepower_ind = 2
cylinders_ind = 0

#Attributes adder
class CustomAttrAdder(BaseEstimator, TransformerMixin):
    def __init__(self, acceleration_on_horsepower = True):
        self.acceleration_on_horsepower = acceleration_on_horsepower
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        acceleration_on_cylinders = X[:, acceleration_ind] / X[:, cylinders_ind]
        if self.acceleration_on_horsepower:
            acceleration_on_horsepower = X[:, acceleration_ind] / X[:, horsepower_ind]
            #np.c_ method concatenate the arrays
            return np.c_[X, acceleration_on_horsepower, acceleration_on_cylinders]
        
        return np.c_[X, acceleration_on_cylinders]

attr_adder = CustomAttrAdder(acceleration_on_horsepower = True)
data_tr_extra_attrs = attr_adder.transform(data_tr.values)
data_tr_extra_attrs[0]
    
    


array([4.0000000e+00, 8.3000000e+01, 6.1000000e+01, 2.0030000e+03,
       1.9000000e+01, 7.4000000e+01, 3.1147541e-01, 4.7500000e+00])

# Creating Pipeline

In [12]:
from sklearn.pipeline import Pipeline
## Scaling numerical atributes
from sklearn.preprocessing import StandardScaler

numerics = ['float64', 'int64']

num_data = data_tr.select_dtypes(include = numerics)

## Pipeline for numerical attributes
# Impute missing values
# Add atributes
# Scale the data
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('attrs_adder', CustomAttrAdder()),
    ('std_scaler', StandardScaler())
])

num_data_tr = num_pipeline.fit_transform(num_data)
num_data_tr[0]


array([-0.85657842, -1.07804475, -1.15192977, -1.17220298,  1.21586943,
       -0.54436373,  1.70952741,  1.29565517])

In [13]:
data.head()

,Cylinders,Displacements,Horsepower,Weight,Acceleration,Model Year,Origin
145,4,83.0,61.0,2003.0,19.0,74,3
151,4,79.0,67.0,2000.0,16.0,74,2
388,4,156.0,92.0,2585.0,14.5,82,1
48,6,250.0,88.0,3139.0,14.5,71,1
114,4,98.0,90.0,2265.0,15.5,73,2


## Transforming numerical and categorical

In [14]:
from sklearn.compose import ColumnTransformer
num_attrs = list(num_data)
cat_attrs = ['Origin']

##complete pipeline to transform
##both numerical and categorical attributes

full_pipeline1 = ColumnTransformer([
    ('num', num_pipeline, num_attrs),
    ('cat', OneHotEncoder(), cat_attrs),
])

prepared_data = full_pipeline1.fit_transform(data)
prepared_data[0]


array([-0.85657842, -1.07804475, -1.15192977, -1.17220298,  1.21586943,
       -0.54436373,  1.70952741,  1.29565517,  0.        ,  0.        ,
        1.        ])

In [15]:
data.head()

,Cylinders,Displacements,Horsepower,Weight,Acceleration,Model Year,Origin
145,4,83.0,61.0,2003.0,19.0,74,3
151,4,79.0,67.0,2000.0,16.0,74,2
388,4,156.0,92.0,2585.0,14.5,82,1
48,6,250.0,88.0,3139.0,14.5,71,1
114,4,98.0,90.0,2265.0,15.5,73,2


In [16]:
from sklearn.compose import ColumnTransformer
def num_pipeline_transformer(data):
    """
    Function to process numerical transformations
    
    Argument:
        data: original dataframe
    Returns:
        num_attrs: numerical dataframe
        num_pipleline: numerical pipeline object
    """
    numerics = ['float64', 'int64']

    num_attrs = data.select_dtypes(include = numerics)

    num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy = 'median')),
        ('attrs_adder', CustomAttrAdder()),
        ('std_scaler', StandardScaler())
        ])
    return num_attrs, num_pipeline

def pipeline_transformer(data):
    """
    Complete transformation pipeline for both
    numerical and categorical data.
    
    Argument:
        data: original dataframe 
    Returns:
        prepared_data: transformed data, ready to use
    """
    cat_attrs = ["Origin"]
    num_attrs, num_pipeline = num_pipeline_transformer(data)
    full_pipeline = ColumnTransformer([
        ("num", num_pipeline, list(num_attrs)),
        ("cat", OneHotEncoder(), cat_attrs),
        ])
    prepared_data = full_pipeline.fit_transform(data)
    return prepared_data



In [17]:
preprocessed_df = preprocess_origin_col(data)
preprocessed_df.head()

,Cylinders,Displacements,Horsepower,Weight,Acceleration,Model Year,Origin
145,4,83.0,61.0,2003.0,19.0,74,Germany
151,4,79.0,67.0,2000.0,16.0,74,USA
388,4,156.0,92.0,2585.0,14.5,82,India
48,6,250.0,88.0,3139.0,14.5,71,India
114,4,98.0,90.0,2265.0,15.5,73,USA


In [18]:
prepared_data = pipeline_transformer(preprocessed_df)
prepared_data[1]

array([-0.85657842, -1.1174582 , -0.9900351 , -1.17581316,  0.13069829,
       -0.54436373,  0.79867454,  0.666186  ,  0.        ,  0.        ,
        1.        ])

# Machine Learning Models

## Selecting and training models
1. Try different algorithms
2. Evaluata (RMS)
3. Model evaluation using Cross Validation
4. Hypeparameter Tuning (GridSearch CV)
5. Check feature importance
6. Evaluate on the test data
7. Save the model

### 1. Try different algorithms

Linear Regression

In [19]:
# Linear regression
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression() #instance the lin reg class
lin_reg.fit(prepared_data, data_labels)


LinearRegression()

Mean Square Error (MSE)


In [20]:
from sklearn.metrics import mean_squared_error

mpg_predictions = lin_reg.predict(prepared_data)
lin_mse = mean_squared_error(data_labels, mpg_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse


2.959040222576087

Decision Tree

In [21]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(prepared_data, data_labels)

DecisionTreeRegressor()

In [22]:
mpg_predictions = tree_reg.predict(prepared_data)
lin_mse = mean_squared_error(data_labels, mpg_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

0.0

It give us zero error thats too good to be true. Overfitting. 
## Cross Validation

In [23]:
from sklearn.model_selection import cross_val_score

tree_reg_scores = cross_val_score(tree_reg,
                        prepared_data,
                        data_labels,
                        scoring="neg_mean_squared_error",
                        cv = 10)
tree_reg_rmse_scores = np.sqrt(-tree_reg_scores)
tree_reg_rmse_scores

array([3.5828585 , 3.25100946, 3.51492354, 3.14046175, 2.33920392,
       2.87135639, 3.7013511 , 3.69628192, 4.12443545, 2.51639784])

In [24]:
tree_reg_rmse_scores.mean()

3.2738279856778894

In [25]:
## Cross validation for lineal regression
lin_reg_scores = cross_val_score(lin_reg,
                        prepared_data,
                        data_labels,
                        scoring="neg_mean_squared_error",
                        cv = 10)
lin_reg_rmse_scores = np.sqrt(-lin_reg_scores)
lin_reg_rmse_scores

array([3.43254597, 3.45157629, 3.6621715 , 2.59652976, 2.48023405,
       2.74798115, 3.32524647, 2.42208917, 3.78133275, 2.8573747 ])

In [26]:
lin_reg_rmse_scores.mean()

3.075708179370932

Random forest regressor model

In [27]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(prepared_data, data_labels)
forest_reg_scores = cross_val_score(forest_reg,
                                    prepared_data,
                                    data_labels,
                                    scoring="neg_mean_squared_error",
                                    cv = 10)
forest_reg_rmse_scores = np.sqrt(-forest_reg_scores)
forest_reg_rmse_scores.mean()

2.5811399598637053

Support Vector Machine Regressor

In [28]:
from sklearn.svm import SVR

svm_reg = SVR(kernel = 'linear')
svm_reg.fit(prepared_data, data_labels)
svm_cv_scores = cross_val_score(svm_reg,
                                prepared_data,
                                data_labels,
                                scoring="neg_mean_squared_error",
                                cv = 10)

svm_rmse_scores = np.sqrt(-svm_cv_scores)
svm_rmse_scores.mean()

3.0865916208027717

## Hyperparameter for random forest

In [29]:
from sklearn.model_selection import GridSearchCV

# set of values for the parameters to try
param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2, 4,6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid,
                          scoring='neg_mean_squared_error',
                          return_train_score=True,
                          cv=10,
                          )
grid_search.fit(prepared_data, data_labels)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [30]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [31]:
cv_scores = grid_search.cv_results_

# just to see the result for every group of parameters
for mean_score, params in zip(cv_scores['mean_test_score'], cv_scores['params']):
    print(np.sqrt(-mean_score), params)

3.542993916390072 {'max_features': 2, 'n_estimators': 3}
3.000579811241326 {'max_features': 2, 'n_estimators': 10}
2.8884173716127677 {'max_features': 2, 'n_estimators': 30}
3.1863591675490155 {'max_features': 4, 'n_estimators': 3}
3.0432198733546425 {'max_features': 4, 'n_estimators': 10}
2.7002306863319294 {'max_features': 4, 'n_estimators': 30}
2.9937891765307296 {'max_features': 6, 'n_estimators': 3}
2.764121433364876 {'max_features': 6, 'n_estimators': 10}
2.646738603492051 {'max_features': 6, 'n_estimators': 30}
3.237387506521811 {'max_features': 8, 'n_estimators': 3}
2.8971691185033848 {'max_features': 8, 'n_estimators': 10}
2.6762886737398444 {'max_features': 8, 'n_estimators': 30}
3.2595796917739395 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
3.067261329272739 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
3.2099083203000025 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
3.02661867682722 {'bootstrap': False, 'max_features': 3, 'n_estimat

## Checking feature importance

In [32]:
#getting the feature importances
feature_importances = grid_search.best_estimator_.feature_importances_

extra_attrs = ['acc_on_power', 'acc_on_cyl']
numerics = ['float64', 'int64']
num_attrs = list(data.select_dtypes(include = numerics))

attrs = num_attrs + extra_attrs
sorted(zip(attrs, feature_importances), reverse = True)


[('acc_on_power', 0.05706646534048758),
 ('acc_on_cyl', 0.003436111298458701),
 ('Weight', 0.20956955249796788),
 ('Origin', 0.03204634113980747),
 ('Model Year', 0.11490253204590081),
 ('Horsepower', 0.12133814113832163),
 ('Displacements', 0.30424319686385015),
 ('Cylinders', 0.13598406458136103),
 ('Acceleration', 0.016187968676307046)]

## Evaluating the system on Test Data

In [33]:
final_model = grid_search.best_estimator_

## for_test 
test = test_set.drop("MPG", axis = 1)
test_labels = test_set["MPG"].copy()

prepared_test = full_pipeline1.fit_transform(test)

final_predictions = final_model.predict(prepared_test)
final_mse = mean_squared_error(test_labels, final_predictions)
final_rmse = np.sqrt(final_mse)


In [34]:
final_rmse

3.1054401414650115

## Function to complete the flow

In [35]:
def predict_mpg(config, model):
    if type(config) == dict:
        df = pd.DataFrame(config)
    else:
        df = config

    preproc_df = preprocess_origin_col(df)
    prepared_df = pipeline_transformer(preproc_df)
    y_pred = model.predict(prepared_df)
    return y_pred
    

In [36]:
vehicle_config = {
    'Cylinders': [4, 6, 8],
    'Displacement': [155.0, 160.0, 165.5],
    'Horsepower': [93.0, 130.0, 98.0],
    'Weight': [2500.0, 3150.0, 2600.0],
    'Acceleration': [15.0, 14.0, 16.0],
    'Model Year': [81, 80, 78],
    'Origin': [3, 2, 1]
}

predict_mpg(vehicle_config, final_model)

array([33.12      , 18.13      , 19.49666667])

## Saving the model

In [37]:
import pickle

In [38]:
with open('model_mpg.bin', 'wb') as f_out:
    pickle.dump(final_model, f_out)
    f_out.close()

## Heroku
Requesting a evaluation of the data from the model deployed in heroku

In [40]:

import requests


url = "https://prediction-mpg123.herokuapp.com/"
r = requests.post(url, json = vehicle_config)
r.text.strip()

'{"mpg_predictions":[32.806666666666665,17.373333333333335,19.959999999999997]}'